<a href="https://colab.research.google.com/github/darioLabrador/computer_vision_practice/blob/main/CSC3831_Practical_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC3831 - Practical 2
Hello Class, welcome to the second practical session. For this practical please read the information in the text blocks, run the code blocks in sequence, and attempt the optional challenges if you have time.

In this part of the course we will use a few Python packages. The packages used are as follows:
- [Pandas](https://pandas.pydata.org/): Data manipulation and analysis tool
- [RecordLinkage](https://recordlinkage.readthedocs.io/en/latest/): A python Record Linkage toolset
- [FuzzyMatcher](https://github.com/RobinL/fuzzymatcher): Legacy record linkage tool, no longer supported
- [MissingNo](https://github.com/ResidentMario/missingno): Graphical tool for easily visualising missing data
- [Numpy](https://numpy.org/): Numerical computing tool
-

Additonally we will be playing around with some more Pandas DataFrame manipulations. Not every method is explained in detail, so if you are interested or want to know more please look at the Pandas method documentation.

The data we will be using today are `hospital_accounts` which contains account information for various hospitals in the USA, and `hospital_reimbursement` which contains reimbursement financial data for various hospitals in the USA.


# Record Linkage

Firstly let's install the matching tools we'll use in today's practical.

In [1]:
!pip install recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.9/355.9 kB 20.8 MB/s eta 0:00:00


In [2]:
!pip install fuzzymatcher

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.3 MB/s eta 0:00:00
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13901 sha256=4bea7a1b505428f45d2d44cc6fdaa63fd955ba65bd516ee60d1fe7e7a22dc637
  Stored in directory: /root/.cache/pip/wheels/d0/fc/72/0c9e56e8c85dc1b2998f4c8f5e9075b8eb71b046b519aa1759
Successfully built metaphone


Now let's load the standard Pandas package and the datasets we're looking to join together.

We want to combine together the two datasets, but they don't have numeric indices that correspond. So how do we go about doing this?

In [3]:
import pandas as pd

In [4]:
hospital_accounts = pd.read_csv('https://github.com/chris1610/pbpython/raw/master/data/hospital_account_info.csv')
hospital_reimbursement = pd.read_csv('https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_reimbursement.csv')

In [5]:
hospital_accounts.head()

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
0,10605,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private
1,24250,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary
2,10341,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private
3,81101,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary
4,39835,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary


In [6]:
hospital_reimbursement.head()

,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52
1,519118,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,43,13289.09,5413.63,4490.93
2,733073,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,73,22261.60,4922.18,4021.79
3,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17
4,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53


The "Account_Num" and "Provider_Num" are indices to be used within the files themselves, but don't nicely match up across files, making this a Record Linkage problem. It looks like there are two name columns, "Facility Name" and "Provider Name" with which we should be able to identify if two records represent the same real-world entity.

Both files use the same spacing and capitilisation format throughout each column so we don't need to do any pre-processing.

Lets see how many records exist in each dataset.

In [7]:
print("raw dataframe number of records:")
print("hospital_accounts:  {x}".format(x=hospital_accounts['Account_Num'].count()))
print("hospital_reimbursement:  {x}".format(x=hospital_reimbursement['Provider_Num'].count()))

raw dataframe number of records:
hospital_accounts:  5339
hospital_reimbursement:  2697


It looks like the `hospital_reimbursements` DataFrame is has less records, so we can try joining based on its keys.

First we just explore the data and then try exact joins.
Focusing on "Facility Name" and "Provider Name" we'll just pick providers starting with "AD" from both (to look at a few examples), and pick the top 20 to visually compare.

In [8]:
cond_accounts = hospital_accounts['Facility Name'].str.startswith("AD")
hospital_accounts.loc[cond_accounts].sort_values(by=['Facility Name'], ascending=True)[['Facility Name', 'Address', 'City']].head(20)

,Facility Name,Address,City
3282,"AD HOSPITAL EAST, LLC","12950 EAST FREEWAY, SUITE 100",HOUSTON
736,ADAIR ACUTE CARE AT OSAWATOMIE STATE HOSPITAL,500 STATE HOSPITAL DRIVE,OSAWATOMIE
2515,ADAIR COUNTY MEMORIAL HOSPITAL,609 SE KENT,GREENFIELD
3915,ADAMS COUNTY REGIONAL MEDICAL CENTER,230 MEDICAL CENTER DRIVE,SEAMAN
280,ADAMS MEMORIAL HOSPITAL,1100 MERCER AVE,DECATUR
3017,ADCARE HOSPITAL OF WORCESTER INC,107 LINCOLN STREET,WORCESTER
2883,ADENA PIKE MEDICAL CENTER,100 DAWN LANE,WAVERLY
4211,ADENA REGIONAL MEDICAL CENTER,272 HOSPITAL ROAD,CHILLICOTHE
4887,ADIRONDACK MEDICAL CENTER - SARANAC LAKE,"2233 STATE ROUTE 86, PO BOX 471",SARANAC LAKE
4429,ADMIN DE SERVICIOS MEDICOS PUERTO RIC,BO MONACILLO CARR NUM 22,SAN JUAN


In [9]:
cond_reimbursement = hospital_reimbursement['Provider Name'].str.startswith('AD')
hospital_reimbursement.loc[cond_reimbursement].sort_values(by=['Provider Name'], ascending=True)[['Provider Name','Provider Street Address', 'Provider City']].head(20)

,Provider Name,Provider Street Address,Provider City
1883,ADENA REGIONAL MEDICAL CENTER,272 HOSPITAL ROAD,CHILLICOTHE
1613,ADIRONDACK MEDICAL CENTER,2233 STATE ROUTE 86,SARANAC LAKE
841,ADVENTIST BOLINGBROOK HOSPITAL,500 REMINGTON BOULEVARD,BOLINGBROOK
840,ADVENTIST GLENOAKS,701 WINTHROP AVENUE,GLENDALE HEIGHTS
1150,ADVENTIST HEALTHCARE SHADY GROVE MEDICAL CENTER,9901 MEDICAL CENTER DRIVE,ROCKVILLE
1124,ADVENTIST HEALTHCARE WASHINGTON ADVENTIST HOSP...,7600 CARROLL AVENUE,TAKOMA PARK
774,ADVENTIST HINSDALE HOSPITAL,120 NORTH OAK ST,HINSDALE
752,ADVENTIST LA GRANGE MEMORIAL HOSPITAL,5101 S WILLOW SPRINGS RD,LA GRANGE
204,ADVENTIST MEDICAL CENTER,115 MALL DRIVE,HANFORD
1985,ADVENTIST MEDICAL CENTER,10123 SE MARKET STREET,PORTLAND


We can see there's some overlap in "Facility Name" and "Provider Name", so a join should be possible.

To begin with, lets try an outer join on the "Facility Name" And "Provider Name" columns.

The [merge](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) method can join two DataFrame objects together using the normal SQL joins. We specifcy two columns from each DataFrame to join on (i.e. to match) and which DataFrame's keys to use.

In [10]:
merged = hospital_accounts.merge(hospital_reimbursement, left_on = 'Facility Name', right_on='Provider Name',  how='right')
merged.head(20)

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,46115.0,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301.0,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,519118,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,43,13289.09,5413.63,4490.93
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,733073,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,73,22261.60,4922.18,4021.79
3,51563.0,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467.0,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17
4,56456.0,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235.0,JEFFERSON,(205) 838-3122,Acute Care Hospitals,Voluntary non-profit - Private,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53
5,88941.0,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968.0,DE KALB,(256) 845-3150,Acute Care Hospitals,Proprietary,785513,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,19,28633.79,5378.53,4635.79
6,16452.0,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007.0,SHELBY,(205) 620-8100,Acute Care Hospitals,Voluntary non-profit - Private,890851,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,95,33169.57,5596.78,4535.48
7,29516.0,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660.0,COLBERT,(256) 386-4556,Acute Care Hospitals,Government - Hospital District or Authority,227327,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660,92,19698.71,5063.65,4224.20
8,65285.0,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360.0,DALE,(334) 774-2601,Acute Care Hospitals,Government - Hospital District or Authority,881894,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360,15,10025.87,5704.73,4592.20
9,45584.0,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116.0,MONTGOMERY,(334) 288-2100,Acute Care Hospitals,Government - Hospital District or Authority,716815,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,168,20290.68,6362.93,5509.19


Just looking at the top 20 records, we can see that rows 1, 2, and 15 have NaN values in several key columns. This implies there are records that exist in the right DataFrame (`hospital_reimbursements`) which don't appear in the left (`hospital_accounts`). If that is true, then there should be more columsn that are exclusive to the right DataFrame in the merged result.


In [11]:
merged.count()

,0
Account_Num,2739
Facility Name,2739
Address,2739
City,2739
State,2739
ZIP Code,2739
County Name,2739
Phone Number,2739
Hospital Type,2739
Hospital Ownership,2739


Removing these incomplete rows allows us to curate a complete dataset for further analysis, and as in lieu of contacting the persons responsible for collecting the `hospital_accounts` data and asking for the missing hospitals there's no way to impute, it is appropriate.

In [12]:
merged.dropna(inplace=True)
merged.reset_index(inplace=True)
# Using the .reset_index function moves the old indices to a column called 'index'.
# Repeating this cell multiple times causes it to crash as there's already an 'index' column, so we'll drop it to prevent that.
merged.drop(['index'], axis=1, inplace=True)
merged.head(20)

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,46115.0,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301.0,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52
1,51563.0,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467.0,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17
2,56456.0,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235.0,JEFFERSON,(205) 838-3122,Acute Care Hospitals,Voluntary non-profit - Private,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53
3,88941.0,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968.0,DE KALB,(256) 845-3150,Acute Care Hospitals,Proprietary,785513,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,19,28633.79,5378.53,4635.79
4,16452.0,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007.0,SHELBY,(205) 620-8100,Acute Care Hospitals,Voluntary non-profit - Private,890851,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,95,33169.57,5596.78,4535.48
5,29516.0,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660.0,COLBERT,(256) 386-4556,Acute Care Hospitals,Government - Hospital District or Authority,227327,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660,92,19698.71,5063.65,4224.20
6,65285.0,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360.0,DALE,(334) 774-2601,Acute Care Hospitals,Government - Hospital District or Authority,881894,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360,15,10025.87,5704.73,4592.20
7,45584.0,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116.0,MONTGOMERY,(334) 288-2100,Acute Care Hospitals,Government - Hospital District or Authority,716815,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,168,20290.68,6362.93,5509.19
8,25491.0,JACKSON HOSPITAL & CLINIC INC,1725 PINE STREET,MONTGOMERY,AL,36106.0,MONTGOMERY,(334) 293-8000,Acute Care Hospitals,Voluntary non-profit - Private,187651,JACKSON HOSPITAL & CLINIC INC,1725 PINE STREET,MONTGOMERY,AL,36106,119,25884.71,5625.53,4471.02
9,16052.0,EAST ALABAMA MEDICAL CENTER,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801.0,LEE,(334) 749-3411,Acute Care Hospitals,Government - Hospital District or Authority,895716,EAST ALABAMA MEDICAL CENTER,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,164,12412.13,5615.55,4663.62


Functionally this is no different from performing an outter left join and dropping columns, as the unique rows to the `hospital_accounts` data would be removed as well.

**Optional Challenge**: Try replicating the same thing using outter left joins in the code snippet below.
- If the merged DataFrame is sorted differently, try looking at the Pandas Merge method documentation to figure out why

In [16]:
right_join = hospital_accounts.merge(hospital_reimbursement, left_on = 'Facility Name', right_on='Provider Name',  how='left')
right_join.head(20)

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,10605,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24250,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10341,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81101,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39835,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,48460,INTERMOUNTAIN HOSPITAL,303 NORTH ALLUMBAUGH STREET,BOISE,ID,83704,ADA,(208) 377-8400,Psychiatric,Proprietary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,30098,CHOATE MENTAL HEALTH & DEVELOPMENT CTR,1000 N MAIN ST,ANNA,IL,62906,UNION,(618) 833-5161,Psychiatric,Government - State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,92639,GOODLAND REGIONAL MEDICAL CENTER,220 WEST SECOND STREET,GOODLAND,KS,67735,SHERMAN,(785) 890-3625,Critical Access Hospitals,Government - Local,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,86448,SPRING HARBOR HOSPITAL,123 ANDOVER ROAD,WESTBROOK,ME,4092,CUMBERLAND,(207) 761-2200,Psychiatric,Government - Federal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,28455,EASTERN SHORE HOSPITAL CENTER,POST OFFICE BOX 800,CAMBRIDGE,MD,21613,DORCHESTER,(410) 221-2525,Psychiatric,Government - State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Note**: If you knew that `hospital_reimbursements` has unique records before conducting an outter join, you could save yourself some headache of dropping by conducting an inner join.

In [17]:
# The DataFrames are switched here because inner join perserves the keys of the left DataFrame and this allows the inner join to be displayed the same as the previous codeblocks.
inner_merged = hospital_reimbursement.merge(hospital_accounts, left_on = 'Provider Name', right_on='Facility Name',  how='inner')
inner_merged.head(20)

,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52,46115,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority
1,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17,51563,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private
2,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53,56456,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,JEFFERSON,(205) 838-3122,Acute Care Hospitals,Voluntary non-profit - Private
3,785513,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,19,28633.79,5378.53,4635.79,88941,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,DE KALB,(256) 845-3150,Acute Care Hospitals,Proprietary
4,890851,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,95,33169.57,5596.78,4535.48,16452,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,SHELBY,(205) 620-8100,Acute Care Hospitals,Voluntary non-profit - Private
5,227327,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660,92,19698.71,5063.65,4224.20,29516,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660,COLBERT,(256) 386-4556,Acute Care Hospitals,Government - Hospital District or Authority
6,881894,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360,15,10025.87,5704.73,4592.20,65285,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360,DALE,(334) 774-2601,Acute Care Hospitals,Government - Hospital District or Authority
7,716815,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,168,20290.68,6362.93,5509.19,45584,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,MONTGOMERY,(334) 288-2100,Acute Care Hospitals,Government - Hospital District or Authority
8,187651,JACKSON HOSPITAL & CLINIC INC,1725 PINE STREET,MONTGOMERY,AL,36106,119,25884.71,5625.53,4471.02,25491,JACKSON HOSPITAL & CLINIC INC,1725 PINE STREET,MONTGOMERY,AL,36106,MONTGOMERY,(334) 293-8000,Acute Care Hospitals,Voluntary non-profit - Private
9,895716,EAST ALABAMA MEDICAL CENTER,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,164,12412.13,5615.55,4663.62,16052,EAST ALABAMA MEDICAL CENTER,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,LEE,(334) 749-3411,Acute Care Hospitals,Government - Hospital District or Authority


Removing the incomplete rows is good, but there's still a discrepancy between the number of overlapping rows and the number of rows in the original `hospital_reimbursement` data.


In [18]:
print("Original Count: {x}".format(x=hospital_reimbursement['Provider Name'].count()))
print("Outter Join Count: {x}".format(x=merged['Provider Name'].count()))
print("Inner Join Count: {x}".format(x=inner_merged['Provider Name'].count()))
print("\nUnaccounted For Rows: {x}".format(x=abs(hospital_reimbursement['Provider Name'].count()-merged['Provider Name'].count())))

Original Count: 2697
Outter Join Count: 2739
Inner Join Count: 2739

Unaccounted For Rows: 42


As we have removed unique rows from our merged data, the remaining unaccounted for rows may be:
- Duplicates caused by two entires having overlapping key names
- Actual hospital accounts and practices sharing the same name

**Optional Challenge**: Try isolating the duplicate entires to see this for yourself.

In [24]:
# Find duplicate entries in the merged DataFrame based on 'Provider Name'
duplicates_in_merged = merged[merged['Provider Name'].duplicated(keep=False)]

print("Duplicate entries in the merged DataFrame:")
display(duplicates_in_merged.sort_values(by='Provider Name'))
print("\nNumber of duplicate entries in the merged DataFrame: {x}".format(x=duplicates_in_merged.shape[0]))

Duplicate entries in the merged DataFrame:


,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
2709,45360.0,AURORA MEDICAL CENTER,975 PORT WASHINGTON ROAD,GRAFTON,WI,53024.0,OZAUKEE,(262) 329-1000,Acute Care Hospitals,Voluntary non-profit - Private,544919,AURORA MEDICAL CENTER,975 PORT WASHINGTON ROAD,GRAFTON,WI,53024,46,21277.70,5598.74,4206.85
2708,45360.0,AURORA MEDICAL CENTER,975 PORT WASHINGTON ROAD,GRAFTON,WI,53024.0,OZAUKEE,(262) 329-1000,Acute Care Hospitals,Voluntary non-profit - Private,384586,AURORA MEDICAL CENTER,36500 AURORA DRIVE,SUMMIT,WI,53066,23,16888.30,5848.87,4266.22
2340,53660.0,CITIZENS MEDICAL CENTER,7939 U S HWY 165 SOUTH,COLUMBIA,LA,71418.0,CALDWELL,(318) 649-6106,Acute Care Hospitals,Government - Hospital District or Authority,199595,CITIZENS MEDICAL CENTER,2701 HOSPITAL DRIVE,VICTORIA,TX,77901,59,20201.12,5231.76,4520.90
2339,30459.0,CITIZENS MEDICAL CENTER,2701 HOSPITAL DRIVE,VICTORIA,TX,77901.0,VICTORIA,(361) 573-9181,Acute Care Hospitals,Voluntary non-profit - Other,199595,CITIZENS MEDICAL CENTER,2701 HOSPITAL DRIVE,VICTORIA,TX,77901,59,20201.12,5231.76,4520.90
2338,71053.0,CITIZENS MEDICAL CENTER,100 E COLLEGE DRIVE,COLBY,KS,67701.0,THOMAS,(785) 462-7511,Critical Access Hospitals,Voluntary non-profit - Private,199595,CITIZENS MEDICAL CENTER,2701 HOSPITAL DRIVE,VICTORIA,TX,77901,59,20201.12,5231.76,4520.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,46755.0,WILSON MEDICAL CENTER,REECE CAMPUS 2600 OTTAWA ROAD,NEODESHA,KS,66757.0,WILSON,(620) 325-2611,Critical Access Hospitals,Government - Local,350387,WILSON MEDICAL CENTER,1705 S TARBORO ST,WILSON,NC,27893,114,22089.24,6045.84,4882.97
1185,48737.0,YORK HOSPITAL,1001 SOUTH GEORGE STREET,YORK,PA,17403.0,YORK,(717) 851-2345,Acute Care Hospitals,Voluntary non-profit - Private,585848,YORK HOSPITAL,15 HOSPITAL DRIVE,YORK,ME,3909,52,15782.12,5957.98,4581.10
1186,51583.0,YORK HOSPITAL,15 HOSPITAL DRIVE,YORK,ME,3909.0,YORK,(207) 363-4321,Acute Care Hospitals,Voluntary non-profit - Private,585848,YORK HOSPITAL,15 HOSPITAL DRIVE,YORK,ME,3909,52,15782.12,5957.98,4581.10
2092,48737.0,YORK HOSPITAL,1001 SOUTH GEORGE STREET,YORK,PA,17403.0,YORK,(717) 851-2345,Acute Care Hospitals,Voluntary non-profit - Private,888799,YORK HOSPITAL,1001 SOUTH GEORGE STREET,YORK,PA,17403,277,17255.35,7937.52,5831.60



Number of duplicate entries in the merged DataFrame: 616


So how many records have we successfully matched?

In [25]:
merged[merged['Provider Name'].notnull()]['Provider Name'].count()

np.int64(2739)

In the case where the keys in each DataFrame are nicely formatted (no spelling mistakes) performing a direct join like this is possible, but as we've discussed in class this is not always the case.

## Record Linkage
Lets look at the Python [Record Linkage](https://recordlinkage.readthedocs.io/en/latest/) toolkit.



In [26]:
import recordlinkage

We need to create the search space on which records are compared against each other. In class we discussed doing a full comparison (complete pairwise), but also using Blocking and Sorted Neightbourhoods to reduce the complexity. Lets look at how to do all three using the `recordlinkage.Indexer()` (information found [here](https://recordlinkage.readthedocs.io/en/latest/ref-index.html)).

For the `left_on` and `right_on` in `block()` and `sortedneighbourhood()`, we want to use a column to block our data an and to sort our data on, which will create smaller groups in the blocking case and sort the data nicely in the sorted case. Using the "State" column in the data we can separate out the data nicely and it allows us to sort the dataset easily as well.

In [27]:
# Build an indexer to index the datset using different blocking methods
## Full Pairwise comparison
full_indexer = recordlinkage.Index()
full_indexer.full()
## Blocking
block_indexer = recordlinkage.Index()
block_indexer.block(left_on='State', right_on='Provider State')
## Sorted Neighbourhoods
sorted_neigh_index = recordlinkage.Index()
sorted_neigh_index.sortedneighbourhood(left_on='State', right_on='Provider State')

<Index>

In [28]:
full_candidates = full_indexer.index(hospital_accounts, hospital_reimbursement)
print("Full Comparison:  {x}".format(x=len(full_candidates)))
block_candidates = block_indexer.index(hospital_accounts, hospital_reimbursement)
print("Block Comparison:  {x}".format(x=len(block_candidates)))
sorted_neigh_candidates = sorted_neigh_index.index(hospital_accounts, hospital_reimbursement)
print("Sorted Neighbourhood Comparison:  {x}".format(x=len(sorted_neigh_candidates)))

Full Comparison:  14399283
Block Comparison:  475830
Sorted Neighbourhood Comparison:  998860


Looking at our different space reduction techniques we can see that performing a full comparison would result in n * m (5339 * 2697) comparisons whereas blocking and sorted neighbourhoods provide a subset of that search space.

**Note**: If there are a large amount of spelling mistakes use [Sorted Nieghbourhood](https://recordlinkage.readthedocs.io/en/latest/ref-index.html#recordlinkage.index.SortedNeighbourhood) as blocking will fail as it excludes too many records on minor spelling mistakes.





As there aren't a large amount of spelling mistakes (from a cursory glance at the data at least) and blocking results in the least number of comparisons, we will use blocking.

Using the [compare](https://recordlinkage.readthedocs.io/en/latest/ref-compare.html#module-recordlinkage.compare) method we can check if the "City" matches the "Provider City" exactly and if the "Facility Name" matches the "Provider Name", and the "Address" matches the "Prodiver Street Address" using Jaro's similarity set to accept if there is a greater than 85% similarity in both attributes.

**Note**: RecordLinkage has several other string comparison methods implemented, including `jarowinkler` (modification of Jaro similarity), `levenshtein` (edit distance), `damerau_levenshtein` (modification of edit distance), `qgram` (ngram), and `cosine`. The threshold is used for similarity, so for the distance functions (i.e. edit, n-gram) it's 1-distance.

In [29]:
compare = recordlinkage.Compare()
compare.exact('City', 'Provider City', label='City')
compare.string('Facility Name',
               'Provider Name',
               method='jaro',
               threshold=0.85,
               label='Hosp_Name')
compare.string('Address',
               'Provider Street Address',
               method='jaro',
               threshold=0.85,
               label='Hosp_Address')
features = compare.compute(block_candidates, hospital_accounts,
                           hospital_reimbursement)

In [30]:
features.sort_values(["City", "Hosp_Name", "Hosp_Address"], ascending=False)

City  Hosp_Name  Hosp_Address
14   1777     1        1.0           1.0
27   1502     1        1.0           1.0
28   779      1        1.0           1.0
34   2170     1        1.0           1.0
38   1308     1        1.0           1.0
...         ...        ...           ...
5338 1435     0        0.0           0.0
     1436     0        0.0           0.0
     1437     0        0.0           0.0
     1438     0        0.0           0.0
     1439     0        0.0           0.0

[475830 rows x 3 columns]

The compare method returns a 1 if the similarity is greater than the threshold and a 0 otherwise. If all three comparisons were 85% similar, then we can be fairly sure the two records represent the same real-world entity.

The two columns to the left without a title are the indices of these records in the orignal table. By using the reset_index() method we can move these columns into the DataFrame, and manually reset the column title to reflect what it represents.

In [31]:
# Get the potential matches
potential_matches = features[features.sum(axis=1) >= 3].reset_index()
potential_matches.rename(columns={"level_0":"account_index", "level_1": "reimbursement_index"}, inplace=True)
potential_matches.head(20)

,account_index,reimbursement_index,City,Hosp_Name,Hosp_Address
0,14,1777,1,1.0,1.0
1,27,1502,1,1.0,1.0
2,28,779,1,1.0,1.0
3,34,2170,1,1.0,1.0
4,38,1308,1,1.0,1.0
5,40,2407,1,1.0,1.0
6,42,2063,1,1.0,1.0
7,43,997,1,1.0,1.0
8,44,794,1,1.0,1.0
9,47,2044,1,1.0,1.0


Now, lets have a look at a couple records and see if they seem to represent the same thing.

In [32]:
hospital_accounts.loc[14,:]

,14
Account_Num,59519
Facility Name,MARIA PARHAM MEDICAL CENTER
Address,PO BOX 59
City,HENDERSON
State,NC
ZIP Code,27536
County Name,VANCE
Phone Number,(252) 431-3708
Hospital Type,Acute Care Hospitals
Hospital Ownership,Proprietary


In [33]:
hospital_reimbursement.loc[1777,:]

,1777
Provider_Num,603343
Provider Name,MARIA PARHAM MEDICAL CENTER
Provider Street Address,PO BOX 59
Provider City,HENDERSON
Provider State,NC
Provider Zip Code,27536
Total Discharges,70
Average Covered Charges,18270.17
Average Total Payments,6180.2
Average Medicare Payments,4849.11


Record 14 in `hospital_accounts` and `hospital_reimbursement` do both appear to represent the same real-world hospital, lets check out another set of records to see if that's also the case.

In [34]:
hospital_accounts.loc[27,:]

,27
Account_Num,71617
Facility Name,NEWARK BETH ISRAEL MEDICAL CENTER
Address,201 LYONS AVE
City,NEWARK
State,NJ
ZIP Code,7112
County Name,ESSEX
Phone Number,(973) 926-7850
Hospital Type,Acute Care Hospitals
Hospital Ownership,Voluntary non-profit - Private


In [35]:
hospital_reimbursement.loc[1502,:]

,1502
Provider_Num,123841
Provider Name,NEWARK BETH ISRAEL MEDICAL CENTER
Provider Street Address,201 LYONS AVE
Provider City,NEWARK
Provider State,NJ
Provider Zip Code,7112
Total Discharges,146
Average Covered Charges,70661.73
Average Total Payments,11955.77
Average Medicare Payments,10009.51


Same thing with the second record. If we continue through all fo the records with an aggregate similarity score of 3, we will see that they all seem to correspond to matching entities.

Now lets see what the total number of each score was across all of the comparisons made.

In [36]:
# What are the score totals?
features.sum(axis=1).value_counts().sort_index(ascending=False)

,count
3.0,2299
2.0,489
1.0,8530
0.0,464512


RecordLinkage was able to perfectly match 2299 records, with 489 strong partial matches (should be manually reviewed) 8530 weak partial matches (should be manually reviewed by an intern), and 464512 non matches.

Now that we've come to the end of our RecordLinkage section, I hope you've enjoyed the differences between manually joining tables and performing record linkage.

**Optional Challenge**: If you want an extra challenge, try playing around with the match threshold levels to see how the partial matches change, as well as using other similarity metrics.

## Fuzzymatcher
Just as another example of a record linking tool here is fuzzymatcher. It is unfortunately no longer being supported but I've left this short example in as it was one of the examples used in last year's practical.

In [37]:
import fuzzymatcher
# Columns to match on from the hospital_accounts df
left_on = ["Facility Name", "Address", "City", "State"]
# Columns to match on from the hostpital_reimbursements df
right_on = ["Provider Name", "Provider Street Address", "Provider City", "Provider State"]
# Try link_table first - this may take a while have
linked = fuzzymatcher.link_table(hospital_accounts,
                                 hospital_reimbursement,
                                 left_on,
                                 right_on,
                                 left_id_col='Account_Num',
                                 right_id_col='Provider_Num')
linked.head()

,__id_left,__id_right,match_score,match_rank,Facility Name,Provider Name,Address,Provider Street Address,City,Provider City,State,Provider State
0,10605,643595,-0.746613,1,SAGE MEMORIAL HOSPITAL,TYLER MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,5950 STATE ROUTE 6 WEST,GANADO,TUNKHANNOCK,AZ,PA
1,10605,766793,-0.820139,2,SAGE MEMORIAL HOSPITAL,ANDERSON HOSPITAL,STATE ROUTE 264 SOUTH 191,6800 STATE ROUTE 162,GANADO,MARYVILLE,AZ,IL
2,10605,572806,-0.853538,3,SAGE MEMORIAL HOSPITAL,MERCY HOSPITAL ANDERSON,STATE ROUTE 264 SOUTH 191,7500 STATE ROAD,GANADO,CINCINNATI,AZ,OH
3,10605,489749,-0.877983,4,SAGE MEMORIAL HOSPITAL,MONTROSE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,800 S 3RD ST,GANADO,MONTROSE,AZ,CO
4,10605,264631,-0.890167,5,SAGE MEMORIAL HOSPITAL,JONES MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,191 NORTH MAIN STREET,GANADO,WELLSVILLE,AZ,NY


In [38]:
linked[linked['Facility Name'].str.contains('GORDON')].sort_values(by='match_score', ascending=False)

,__id_left,__id_right,match_score,match_rank,Facility Name,Provider Name,Address,Provider Street Address,City,Provider City,State,Provider State
310640,24125,515652,0.986440,1,ADVENTHEALTH GORDON,GORDON HOSPITAL,1035 RED BUD ROAD,1035 RED BUD ROAD,CALHOUN,CALHOUN,GA,GA
309991,24261,693188,-0.254650,1,GORDON MEMORIAL HOSPITAL DISTRICT,STRINGFELLOW MEMORIAL HOSPITAL,300 EAST 8TH ST,301 EAST 18TH ST,GORDON,ANNISTON,NE,AL
309992,24261,421579,-0.573855,2,GORDON MEMORIAL HOSPITAL DISTRICT,COLUMBUS COMMUNITY HOSPITAL,300 EAST 8TH ST,4600 38TH ST,GORDON,COLUMBUS,NE,NE
309993,24261,340833,-0.615630,3,GORDON MEMORIAL HOSPITAL DISTRICT,RIDEOUT MEMORIAL HOSPITAL,300 EAST 8TH ST,726 4TH ST,GORDON,MARYSVILLE,NE,CA
309994,24261,522917,-0.654419,4,GORDON MEMORIAL HOSPITAL DISTRICT,BAKERSFIELD MEMORIAL HOSPITAL,300 EAST 8TH ST,420 34TH ST BOX,GORDON,BAKERSFIELD,NE,CA
...,...,...,...,...,...,...,...,...,...,...,...,...
310202,24261,875973,-1.751271,212,GORDON MEMORIAL HOSPITAL DISTRICT,ST ELIZABETH COMMUNITY HOSPITAL,300 EAST 8TH ST,2550 SISTER MARY COLUMBA DRIVE,GORDON,RED BLUFF,NE,CA
310203,24261,236029,-1.803042,213,GORDON MEMORIAL HOSPITAL DISTRICT,PEACEHEALTH SOUTHWEST MEDICAL CENTER,300 EAST 8TH ST,400 NE MOTHER JOSEPH PLACE,GORDON,VANCOUVER,NE,WA
310204,24261,683090,-1.820631,214,GORDON MEMORIAL HOSPITAL DISTRICT,CLAY COUNTY HOSPITAL,300 EAST 8TH ST,83825 HIGHWAY 9 P O BOX 1270,GORDON,ASHLAND,NE,AL
310205,24261,198667,-1.922763,215,GORDON MEMORIAL HOSPITAL DISTRICT,LOVELACE MEDICAL CENTER,300 EAST 8TH ST,601 DR MARTIN LUTHER KING JR AVE NE,GORDON,ALBUQUERQUE,NE,NM


We can see that the `match_score` column shows how certain the FuzzyMatcher is that two records represent the same thing. Looking at the first record, we can see that the address from both files is the same, and while the "Facility Name" and "Provider Name" are slightly different, they probably represent the same real-world entity. We wouldn't have been able to make this match just using a single join, so looking across multiple columns like we have here can make better Record Linkage decisions.

Now lets use a `fuzzy_left_join` to combine our two datasets together and see the best and worst matches bewteen the two sets.

In [39]:
# It will take several minutes to run on this data set
matched_results = fuzzymatcher.fuzzy_left_join(hospital_accounts,
                                               hospital_reimbursement,
                                               left_on,
                                               right_on,
                                               left_id_col='Account_Num',
                                               right_id_col='Provider_Num')

In [40]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "Facility Name", "Provider Name", "Address", "Provider Street Address",
    "Provider City", "City", "Provider State", "State"
]

In [41]:
# Let's see the best 5 matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
78296,3.090931,RARITAN BAY MEDICAL CENTER PERTH AMBOY DIVISION,RARITAN BAY MEDICAL CENTER PERTH AMBOY DIVISION,530 NEW BRUNSWICK AVE,530 NEW BRUNSWICK AVE,PERTH AMBOY,PERTH AMBOY,NJ,NJ
533136,2.799072,ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL,ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL,ONE ROBERT WOOD JOHNSON PLACE,ONE ROBERT WOOD JOHNSON PLACE,NEW BRUNSWICK,NEW BRUNSWICK,NJ,NJ
78906,2.785132,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,1325 S CLIFF AVE POST OFFICE BOX 5045,1325 S CLIFF AVE POST OFFICE BOX 5045,SIOUX FALLS,SIOUX FALLS,SD,SD
242735,2.778860,JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFE...,JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFE...,75 NORTH COUNTRY ROAD,75 NORTH COUNTRY ROAD,PORT JEFFERSON,PORT JEFFERSON,NY,NY
447854,2.721425,MAYO CLINIC HEALTH SYSTEM - RED WING,MAYO CLINIC HEALTH SYSTEM IN RED WING,"701 HEWITT BOULEVARD, PO BOX 95","701 HEWITT BOULEVARD, PO BOX 95",RED WING,RED WING,MN,MN


In [42]:
# Now the worst matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=True).head(5)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
426629,-2.268231,CENTRO MEDICO WILMA N VAZQUEZ,BAPTIST MEDICAL CENTER EAST,CARR. 2 KM 39.5 ROAD NUMBER 2 BO ALGARROBO,400 TAYLOR ROAD,MONTGOMERY,VEGA BAJA,AL,PR
83501,-2.124071,DOCTOR CENTER HOSPITAL SAN FERNANDO DE LA CARO...,OVERLAKE HOSPITAL MEDICAL CENTER,EDIF JESUS T PINEIRO AVE FERNANDEZ JUNCOS BO P...,1035-116TH AVE NE,BELLEVUE,CAROLINA,WA,PR
42861,-2.106746,HOSPITAL ONCOLOGICO DR ISAAC GONZALEZ MARTINEZ,SCRIPPS MERCY HOSPITAL,BO. MONACILLOS CARR 22 CENTRO MEDICO DE PUERTO...,4077 5TH AVE,SAN DIEGO,SAN JUAN,CA,PR
450732,-2.050888,CENTRO DE SALUD CONDUCTUAL MENONITA-CIMA,MILFORD REGIONAL MEDICAL CENTER,CARR ESTATAL 14 INTERIOR SARGENTO GERARDO SANT...,14 PROSPECT STREET,MILFORD,AIBONITO,MA,PR
476484,-1.996508,ADMIN DE SERVICIOS MEDICOS PUERTO RIC,MAINE MEDICAL CENTER,BO MONACILLO CARR NUM 22,22 BRAMHALL ST,PORTLAND,SAN JUAN,ME,PR


There is a tool called [Splink](https://moj-analytical-services.github.io/splink/index.html) which is being actively contributed to while FuzzyMatcher is no longer supported. I did not have enough time to create a brief splink tutorial, and the one from last year is depreciated and no longer functions, but if you have the inclination please give it a try in your spare time.